In [1]:
from flask import Flask , request , jsonify
import pyodbc
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId
import threading
import time
from datetime import datetime, date
from decimal import Decimal
import requests


In [2]:
app = Flask(__name__)

# اتصال بالـ MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["MyDatabase"]

source_coll = db["RetailCustomerData_Coll"]
target_coll = db["SalesCollection1"]
target_coll = db["TopCustomers"] 
target_coll = db["BestSellingProducts"] 
target_coll = db["BestSellingProductsByBranch"] 
target_coll = db["BranchRevenueComparison"] 
target_coll = db["MonthlySalesTrends"] 
target_coll = db["SeasonalProductDemand"] 
target_coll = db["StockPlanning"] 

In [ ]:
# ============================
# 1- API to generate SalesCollection from source data
# ============================
@app.route("/generate_sales_collection", methods=["POST"])
def generate_sales_collection():
    try:
        pipeline = [
            {
                "$group": {
                    "_id": {
                        "CustomerID": "$CustomerID",
                        "CustomerName": "$CustomerName",
                        "PreferredBranch": "$BranchName"
                    },
                    "TotalSpent": {"$sum": "$TotalPrice"},
                    "TotalOrders": {"$addToSet": "$SaleID"},
                    "TopProducts": {
                        "$push": {
                            "ProductName": "$ProductName",
                            "Quantity": "$Quantity"
                        }
                    },
                    "MonthlyPurchases": {
                        "$push": {
                            "Month": {
                                "$dateToString": {"format": "%Y-%m", "date": "$SaleDate"}
                            },
                            "Amount": "$TotalPrice"
                        }
                    }
                }
            },
            {
                "$project": {
                    "_id": 0,
                    "CustomerID": "$_id.CustomerID",
                    "CustomerName": "$_id.CustomerName",
                    "PreferredBranch": "$_id.PreferredBranch",
                    "TotalSpent": 1,
                    "TotalOrders": {"$size": "$TotalOrders"},
                    "TopProducts": 1,
                    "MonthlyPurchases": {
                        "$arrayToObject": {
                            "$map": {
                                "input": "$MonthlyPurchases",
                                "as": "mp",
                                "in": {"k": "$$mp.Month", "v": "$$mp.Amount"}
                            }
                        }
                    }
                }
            },
            {"$out": "SalesCollection1"}
        ]

        # Execute aggregation pipeline
        source_coll.aggregate(pipeline)

        return jsonify({"message": "SalesCollection generated successfully ✅"}), 201
    except Exception as e:
        return jsonify({"error": str(e)}), 500


# ============================
# 2- API to fetch data from SalesCollection
# ============================
@app.route("/get_sales_collection", methods=["GET"])
def get_sales_collection():
    try:
        data = list(target_coll.find({}, {"_id": 0}))
        return jsonify(data), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500


In [4]:

# ============================
# API: Generate Top Customers Collection
# ============================
@app.route("/generate_top_customers", methods=["POST"])
def generate_top_customers():
    try:
        pipeline = [
            {
                "$group": {
                    "_id": {
                        "CustomerID": "$CustomerID",
                        "CustomerName": "$CustomerName"
                    },
                    "TotalSpent": {"$sum": "$TotalPrice"}
                }
            },
            {
                "$project": {
                    "_id": 0,
                    "CustomerID": "$_id.CustomerID",
                    "CustomerName": "$_id.CustomerName",
                    "TotalSpent": 1
                }
            },
            {"$sort": {"TotalSpent": -1}},   # Sort descending
            {"$limit": 5},                   # Top 5
            {"$out": "TopCustomers"}         # Save result in new collection
        ]

        source_coll.aggregate(pipeline)
        return jsonify({"message": "TopCustomers collection created successfully ✅"}), 201

    except Exception as e:
        return jsonify({"error": str(e)}), 500


# ============================
# API: Fetch Top Customers Collection
# ============================
@app.route("/get_top_customers", methods=["GET"])
def get_top_customers():
    try:
        data = list(target_coll.find({}, {"_id": 0}))
        return jsonify(data), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [5]:
# ============================
# API: Generate Best-Selling Products Collection
# ============================
@app.route("/generate_best_selling_products", methods=["POST"])
def generate_best_selling_products():
    try:
        pipeline = [
            {
                "$group": {
                    "_id": "$ProductName",
                    "TotalQuantity": {"$sum": "$Quantity"},
                    "TotalRevenue": {"$sum": "$TotalPrice"}
                }
            },
            {
                "$project": {
                    "_id": 0,
                    "ProductName": "$_id",
                    "TotalQuantity": 1,
                    "TotalRevenue": 1
                }
            },
            {"$sort": {"TotalQuantity": -1}},   # Sort by quantity sold (descending)
            {"$limit": 10},                     # Top 10 best-selling products
            {"$out": "BestSellingProducts"}     # Save in new collection
        ]

        source_coll.aggregate(pipeline)
        return jsonify({"message": "BestSellingProducts collection created successfully ✅"}), 201

    except Exception as e:
        return jsonify({"error": str(e)}), 500


# ============================
# API: Fetch Best-Selling Products Collection
# ============================
@app.route("/get_best_selling_products", methods=["GET"])
def get_best_selling_products():
    try:
        data = list(db["BestSellingProducts"].find({}, {"_id": 0}))
        return jsonify(data), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [6]:
# ============================
# API: Generate Best-Selling Products by Branch Collection
# ============================
@app.route("/generate_best_selling_products_by_branch", methods=["POST"])
def generate_best_selling_products_by_branch():
    try:
        pipeline = [
            {
                "$group": {
                    "_id": {
                        "BranchName": "$BranchName",
                        "ProductName": "$ProductName"
                    },
                    "TotalQuantity": {"$sum": "$Quantity"},
                    "TotalRevenue": {"$sum": "$TotalPrice"}
                }
            },
            {
                "$project": {
                    "_id": 0,
                    "BranchName": "$_id.BranchName",
                    "ProductName": "$_id.ProductName",
                    "TotalQuantity": 1,
                    "TotalRevenue": 1
                }
            },
            {"$sort": {"BranchName": 1, "TotalQuantity": -1}},  # Sort by branch then quantity
            {"$out": "BestSellingProductsByBranch"}             # Save in new collection
        ]

        source_coll.aggregate(pipeline)
        return jsonify({"message": "BestSellingProductsByBranch collection created successfully ✅"}), 201

    except Exception as e:
        return jsonify({"error": str(e)}), 500


# ============================
# API: Fetch Best-Selling Products by Branch Collection
# ============================
@app.route("/get_best_selling_products_by_branch", methods=["GET"])
def get_best_selling_products_by_branch():
    try:
        data = list(db["BestSellingProductsByBranch"].find({}, {"_id": 0}))
        return jsonify(data), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500


In [ ]:
# ============================
# API: Generate Branch Revenue Comparison Collection
# ============================
@app.route("/generate_branch_revenue_comparison", methods=["POST"])
def generate_branch_revenue_comparison():
    try:
        pipeline = [
            {
                "$group": {
                    "_id": "$BranchName",
                    "TotalRevenue": {"$sum": "$TotalPrice"}
                }
            },
            {
                "$project": {
                    "_id": 0,
                    "BranchName": "$_id",
                    "TotalRevenue": 1
                }
            },
            {"$sort": {"TotalRevenue": -1}},  
            {"$out": "BranchRevenueComparison"}
        ]

        source_coll.aggregate(pipeline)
        return jsonify({"message": "BranchRevenueComparison collection created successfully ✅"}), 201

    except Exception as e:
        return jsonify({"error": str(e)}), 500


# ============================
# API: Fetch Branch Revenue Comparison Collection
# ============================
@app.route("/get_branch_revenue_comparison", methods=["GET"])
def get_branch_revenue_comparison():
    try:
        data = list(db["BranchRevenueComparison"].find({}, {"_id": 0}))
        return jsonify(data), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500


In [8]:
# ============================
# API: Generate Monthly Sales Trends Collection
# ============================
@app.route("/generate_monthly_sales_trends", methods=["POST"])
def generate_monthly_sales_trends():
    try:
        pipeline = [
            {
                "$group": {
                    "_id": {
                        "Month": {"$dateToString": {"format": "%Y-%m", "date": "$SaleDate"}}
                    },
                    "TotalRevenue": {"$sum": "$TotalPrice"}
                }
            },
            {
                "$project": {
                    "_id": 0,
                    "Month": "$_id.Month",
                    "TotalRevenue": 1
                }
            },
            {"$sort": {"Month": 1}},  # ترتيب تصاعدي بالشهور
            {"$out": "MonthlySalesTrends"}  # حفظ النتيجة في كوليكشن جديد
        ]

        source_coll.aggregate(pipeline)
        return jsonify({"message": "MonthlySalesTrends collection created successfully ✅"}), 201

    except Exception as e:
        return jsonify({"error": str(e)}), 500


# ============================
# API: Fetch Monthly Sales Trends Collection
# ============================
@app.route("/get_monthly_sales_trends", methods=["GET"])
def get_monthly_sales_trends():
    try:
        data = list(db["MonthlySalesTrends"].find({}, {"_id": 0}))
        return jsonify(data), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500


In [9]:
# ============================
# API: Generate Seasonal Product Demand Collection
# ============================
@app.route("/generate_seasonal_product_demand", methods=["POST"])
def generate_seasonal_product_demand():
    try:
        pipeline = [
            {
                "$group": {
                    "_id": {
                        "Month": {"$dateToString": {"format": "%Y-%m", "date": "$SaleDate"}},
                        "ProductName": "$ProductName"
                    },
                    "TotalQuantity": {"$sum": "$Quantity"},
                    "TotalRevenue": {"$sum": "$TotalPrice"}
                }
            },
            {
                "$project": {
                    "_id": 0,
                    "Month": "$_id.Month",
                    "ProductName": "$_id.ProductName",
                    "TotalQuantity": 1,
                    "TotalRevenue": 1
                }
            },
            {"$sort": {"Month": 1, "TotalQuantity": -1}},
            {"$out": "SeasonalProductDemand"}
        ]

        source_coll.aggregate(pipeline)
        return jsonify({"message": "SeasonalProductDemand collection created successfully ✅"}), 201

    except Exception as e:
        return jsonify({"error": str(e)}), 500


# ============================
# API: Fetch Seasonal Product Demand Collection
# ============================
@app.route("/get_seasonal_product_demand", methods=["GET"])
def get_seasonal_product_demand():
    try:
        data = list(db["SeasonalProductDemand"].find({}, {"_id": 0}))
        return jsonify(data), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500


In [10]:
# ============================
# API: Generate Stock Planning Collection
# ============================
@app.route("/generate_stock_planning", methods=["POST"])
def generate_stock_planning():
    try:
        pipeline = [
            {
                "$group": {
                    "_id": {
                        "BranchName": "$BranchName",
                        "ProductName": "$ProductName"
                    },
                    "TotalQuantity": {"$sum": "$Quantity"}
                }
            },
            {
                "$project": {
                    "_id": 0,
                    "BranchName": "$_id.BranchName",
                    "ProductName": "$_id.ProductName",
                    "TotalQuantity": 1
                }
            },
            {"$sort": {"BranchName": 1, "TotalQuantity": -1}},
            {"$out": "StockPlanning"}
        ]

        source_coll.aggregate(pipeline)
        return jsonify({"message": "StockPlanning collection created successfully ✅"}), 201

    except Exception as e:
        return jsonify({"error": str(e)}), 500


# ============================
# API: Fetch Stock Planning Collection
# ============================
@app.route("/get_stock_planning", methods=["GET"])
def get_stock_planning():
    try:
        data = list(db["StockPlanning"].find({}, {"_id": 0}))
        return jsonify(data), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500


In [ ]:
# Code to Run the Flask API 
def run_flask():
    app.run(port=5000, debug=False, use_reloader=False)

# Code to Run the Flask API in a separate thread in the background  
threading.Thread(target=run_flask).start()
time.sleep(1)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [27/Aug/2025 23:55:08] "POST /generate_sales_collection HTTP/1.1" 201 -
127.0.0.1 - - [27/Aug/2025 23:55:08] "GET /get_sales_collection HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2025 23:55:08] "POST /generate_top_customers HTTP/1.1" 201 -
127.0.0.1 - - [27/Aug/2025 23:55:08] "GET /get_top_customers HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2025 23:55:08] "POST /generate_best_selling_products HTTP/1.1" 201 -
127.0.0.1 - - [27/Aug/2025 23:55:08] "GET /get_best_selling_products HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2025 23:55:08] "POST /generate_best_selling_products_by_branch HTTP/1.1" 201 -
127.0.0.1 - - [27/Aug/2025 23:55:08] "GET /get_best_selling_products_by_branch HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2025 23:55:08] "POST /generate_branch_revenue_comparison HTTP/1.1" 201 -
127.0.0.1 - - [27/Aug/2025 23:55:08] "POST /generate_monthly_sales_trends HTTP/1.1" 201 -
127.0.0.1 - - [27/Aug/2025 23:55:08] "POST /generate_seasonal_product_demand HTTP/1.1" 201 -
127.0.0.1 - - [27/Aug/20

In [12]:
import requests

url = "http://127.0.0.1:5000/generate_sales_collection"

response = requests.post(url)

print("Status Code:", response.status_code)
print("Response:", response.json())


Status Code: 201
Response: {'message': 'SalesCollection generated successfully ✅'}


In [13]:
import requests

url = "http://127.0.0.1:5000/get_sales_collection"

response = requests.get(url)

print("Status Code:", response.status_code)
print("Response:", response.json())


Status Code: 200
Response: []


In [14]:
url = "http://127.0.0.1:5000/generate_top_customers"

response = requests.post(url)

print("Status Code:", response.status_code)
print("Response:", response.json())

Status Code: 201
Response: {'message': 'TopCustomers collection created successfully ✅'}


In [15]:
url = "http://127.0.0.1:5000/get_top_customers"

response = requests.get(url)

print("Status Code:", response.status_code)
print("Response:", response.json())


Status Code: 200
Response: []


In [16]:
url = "http://127.0.0.1:5000/generate_best_selling_products"

response = requests.post(url)

print("Status Code:", response.status_code)
print("Response:", response.json())

Status Code: 201
Response: {'message': 'BestSellingProducts collection created successfully ✅'}


In [17]:
url = "http://127.0.0.1:5000/get_best_selling_products"

response = requests.get(url)

print("Status Code:", response.status_code)
print("Response:", response.json())


Status Code: 200
Response: [{'ProductName': 'Ball', 'TotalQuantity': 184, 'TotalRevenue': 159047.41999999998}, {'ProductName': 'Poor', 'TotalQuantity': 143, 'TotalRevenue': 160964.72999999998}, {'ProductName': 'Price', 'TotalQuantity': 135, 'TotalRevenue': 102053.15000000001}, {'ProductName': 'Kid', 'TotalQuantity': 121, 'TotalRevenue': 148602.14}, {'ProductName': 'Seem', 'TotalQuantity': 120, 'TotalRevenue': 129247.12}, {'ProductName': 'Every', 'TotalQuantity': 119, 'TotalRevenue': 196156.09000000003}, {'ProductName': 'Best', 'TotalQuantity': 117, 'TotalRevenue': 142941.98}, {'ProductName': 'Enter', 'TotalQuantity': 115, 'TotalRevenue': 147427.89}, {'ProductName': 'Heart', 'TotalQuantity': 112, 'TotalRevenue': 111535.08}, {'ProductName': 'Suddenly', 'TotalQuantity': 110, 'TotalRevenue': 180789.98}]


In [18]:
url = "http://127.0.0.1:5000/generate_best_selling_products_by_branch"

response = requests.post(url)

print("Status Code:", response.status_code)
print("Response:", response.json())

Status Code: 201
Response: {'message': 'BestSellingProductsByBranch collection created successfully ✅'}


In [19]:
url = "http://127.0.0.1:5000/get_best_selling_products_by_branch"

response = requests.get(url)

print("Status Code:", response.status_code)
print("Response:", response.json())


Status Code: 200
Response: [{'BranchName': 'Abbott, Moore and Larson', 'ProductName': 'Paper', 'TotalQuantity': 22, 'TotalRevenue': 6382.42}, {'BranchName': 'Adams Group', 'ProductName': 'Prove', 'TotalQuantity': 36, 'TotalRevenue': 35870.04}, {'BranchName': 'Adams Group', 'ProductName': 'Professional', 'TotalQuantity': 10, 'TotalRevenue': 14442.8}, {'BranchName': 'Adams, Russell and Hardin', 'ProductName': 'Kid', 'TotalQuantity': 24, 'TotalRevenue': 47900.16}, {'BranchName': 'Adams, Russell and Hardin', 'ProductName': 'Exactly', 'TotalQuantity': 7, 'TotalRevenue': 4674.46}, {'BranchName': 'Adams, Russell and Hardin', 'ProductName': 'Instead', 'TotalQuantity': 6, 'TotalRevenue': 11315.7}, {'BranchName': 'Ali PLC', 'ProductName': 'Need', 'TotalQuantity': 36, 'TotalRevenue': 26415.72}, {'BranchName': 'Ali PLC', 'ProductName': 'Fight', 'TotalQuantity': 6, 'TotalRevenue': 1169.28}, {'BranchName': 'Allen-Harris', 'ProductName': 'With', 'TotalQuantity': 12, 'TotalRevenue': 8338.8}, {'BranchN

In [20]:
url = "http://127.0.0.1:5000/generate_branch_revenue_comparison"

response = requests.post(url)

print("Status Code:", response.status_code)
print("Response:", response.json())


url = "http://127.0.0.1:5000/generate_monthly_sales_trends"

response = requests.post(url)

print("Status Code:", response.status_code)
print("Response:", response.json())


url = "http://127.0.0.1:5000/generate_seasonal_product_demand"

response = requests.post(url)

print("Status Code:", response.status_code)
print("Response:", response.json())


url = "http://127.0.0.1:5000/generate_stock_planning"

response = requests.post(url)

print("Status Code:", response.status_code)
print("Response:", response.json())

Status Code: 201
Response: {'message': 'BranchRevenueComparison collection created successfully ✅'}
Status Code: 201
Response: {'message': 'MonthlySalesTrends collection created successfully ✅'}
Status Code: 201
Response: {'message': 'SeasonalProductDemand collection created successfully ✅'}
Status Code: 201
Response: {'message': 'StockPlanning collection created successfully ✅'}


In [21]:
url = "http://127.0.0.1:5000/get_branch_revenue_comparison"

response = requests.get(url)

print("Status Code:", response.status_code)
print("Response:", response.json())


url = "http://127.0.0.1:5000/get_monthly_sales_trends"

response = requests.get(url)

print("Status Code:", response.status_code)
print("Response:", response.json())


url = "http://127.0.0.1:5000/get_seasonal_product_demand"

response = requests.get(url)

print("Status Code:", response.status_code)
print("Response:", response.json())


url = "http://127.0.0.1:5000/get_stock_planning"

response = requests.get(url)

print("Status Code:", response.status_code)
print("Response:", response.json())


Status Code: 200
Response: [{'BranchName': 'Rice Ltd', 'TotalRevenue': 257524.56}, {'BranchName': 'Flores, Griffin and Thompson', 'TotalRevenue': 230370.56}, {'BranchName': 'Villanueva Ltd', 'TotalRevenue': 198518.22}, {'BranchName': 'Vasquez and Sons', 'TotalRevenue': 179681.18}, {'BranchName': 'Sanders PLC', 'TotalRevenue': 166962.74}, {'BranchName': 'Schroeder-Davenport', 'TotalRevenue': 166276.34}, {'BranchName': 'Russell, Church and Huang', 'TotalRevenue': 162406.36000000002}, {'BranchName': 'City Center Branch', 'TotalRevenue': 162376.8}, {'BranchName': 'Taylor, Fleming and Moody', 'TotalRevenue': 160536.1}, {'BranchName': 'Wagner PLC', 'TotalRevenue': 159727.68}, {'BranchName': 'Hicks-Ramirez', 'TotalRevenue': 148698.79}, {'BranchName': 'Sanchez-Ponce', 'TotalRevenue': 148163.14}, {'BranchName': 'Haney Group', 'TotalRevenue': 140461.83}, {'BranchName': 'Chavez-Andrews', 'TotalRevenue': 140004.38}, {'BranchName': 'Kelly-Hall', 'TotalRevenue': 139974.28}, {'BranchName': 'Kim Ltd',